In [28]:
import pandas as pd
import requests
import re
import click
import csv

In [29]:
class Book:
    def __init__(self, title, author, isbn, publisher, data_read, my_rating, media):
        self.title = title
        self.author = author
        self.isbn = isbn
        self.my_rating = my_rating
        self.average_rating = ''
        self.publisher = publisher
        self.binding = ''
        self.year_published = ''
        self.original_publication_year = ''
        self.data_read = data_read
        self.data_added = ''
        self.bookshelves = ''
        self.my_review = ''
        self.media = media

    def __str__(self):
        return f"{self.title} - {self.author}. (ISBN: {self.isbn})"

In [30]:
def fetch_book_isbn(book_url):
    isbn = 0
    r = requests.get(f"https://www.skoob.com.br{book_url}")

    if r.status_code == 200:
        match = re.search(r'<meta property="books:isbn" content="(\w+)"', r.text)
        if match:
            isbn = match.groups()[0]
    return isbn


def export_to_csv(user, books):
    with open(f'{user}-my-books.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Title', 'Author', 'ISBN', 'My Rating', 'Average Rating', 'Publisher', 'Binding',
                         'Year Published', 'Original Publication Year', 'Date Read', 'Date Added', 'Bookshelves',
                         'My Review', 'Media'])
        for book in books:
            writer.writerow([book.title, book.author, book.isbn, book.my_rating, book.average_rating, book.publisher,
                             book.binding, book.year_published, book.original_publication_year, book.data_read,
                             book.data_added, book.bookshelves, book.my_review, book.media])

In [31]:
"""Skoob books exporter"""
books = []
page = 0
has_next = True
user=8132020
shelf=1 # 0 - Todos, 1 - Lidos, 2 - Lendo, 3 - Quero Ler, 4 - Relendo, 5 - Abandonei
book_json_2 = ""

print("Reading books from Skoob...")
while has_next:
    page += 1
    r = requests.get(f"https://www.skoob.com.br/v1/bookcase/books/{user}/shelf_id:{shelf}/page:{page}/limit:50/")

    if r.status_code == 200:
        json_response = r.json()
        books_response = json_response['response']
        has_next = json_response['paging']['next_page']
        for book_json in books_response:
            book_json_2 = book_json
            isbn = fetch_book_isbn(book_json['edicao']['url'])
            book = Book(book_json['edicao']['titulo'], book_json['edicao']['autor'].split(',')[0], isbn,
                        book_json['edicao']['editora'], book_json['dt_leitura'][:10], book_json['ranking'],
                        book_json['media'])
            books.append(book)
            print(book)

print("\nExporting books to csv...")
export_to_csv(user, books)
print("\nDone!")

Reading books from Skoob...
Tress, A Garota do Mar Esmeralda - Brandon Sanderson. (ISBN: 9786589132943)
Imperfeitos - Christina Lauren. (ISBN: 9786559571284)
A Destruidora De Casamentos - Mia Sosa. (ISBN: 9786559702664)
Conquistada pelo Bilionário Italiano - T. M. Kechichian. (ISBN: 0553380958)
Amor, teoricamente - Ali Hazelwood. (ISBN: 9786555655254)
Aliança de Casamento - Jasmine Guillory. (ISBN: 9788542216431)
Le Petit Prince - Antoine de Saint-Exupéry. (ISBN: 2070256626)
A Razão do Amor - Ali Hazelwood. (ISBN: 9786555653991)
ALEXANDER STONE - Cleo Luz. (ISBN: 0553380958)
Segredo lascivo - Natália Dias. (ISBN: 0553380958)
Só Entre Nós Dois - Fernanda Santana. (ISBN: 0553380958)
A Lenda de Ruff Ghanor - Leonel Caldela. (ISBN: 0553380958)
Loucos Por Livros - Emily Henry. (ISBN: 9786559241538)
A Boa Sorte - Álex Rovira Celma. (ISBN: 9788543103259)
Obcecado pela Protegida - D. A. Lemoyne. (ISBN: 0553380958)
O Juiz Bad Boy - Bianca Pohndorf. (ISBN: 0553380958)
Quando o conde encontra seu

In [32]:
# with open('book.json', 'w') as f:
#     f.write(str(book_json_2))

In [33]:
df_books = pd.read_csv('8132020-my-books.csv')
df_books['Date Read'] = pd.to_datetime(df_books['Date Read'], infer_datetime_format=True)

In [34]:
df_books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117 entries, 0 to 116
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Title                      117 non-null    object        
 1   Author                     117 non-null    object        
 2   ISBN                       117 non-null    int64         
 3   My Rating                  117 non-null    float64       
 4   Average Rating             0 non-null      float64       
 5   Publisher                  117 non-null    object        
 6   Binding                    0 non-null      float64       
 7   Year Published             0 non-null      float64       
 8   Original Publication Year  0 non-null      float64       
 9   Date Read                  117 non-null    datetime64[ns]
 10  Date Added                 0 non-null      float64       
 11  Bookshelves                0 non-null      float64       
 12  My Revie

In [40]:
df_books = df_books[df_books['Date Read'] >= '2023-01-01']
df_books.shape

(59, 14)

In [41]:
df_books.head(10)

,Title,Author,ISBN,My Rating,Average Rating,Publisher,Binding,Year Published,Original Publication Year,Date Read,Date Added,Bookshelves,My Review,Media
0,"Tress, A Garota do Mar Esmeralda",Brandon Sanderson,9786589132943,5.0,NaN,Trama,NaN,NaN,NaN,2023-09-15,NaN,NaN,NaN,0
1,Imperfeitos,Christina Lauren,9786559571284,3.5,NaN,Faro Editorial,NaN,NaN,NaN,2023-09-15,NaN,NaN,NaN,2
2,A Destruidora De Casamentos,Mia Sosa,9786559702664,4.0,NaN,Harlequin,NaN,NaN,NaN,2023-08-30,NaN,NaN,NaN,0
3,Conquistada pelo Bilionário Italiano,T. M. Kechichian,553380958,2.5,NaN,Independente,NaN,NaN,NaN,2023-08-27,NaN,NaN,NaN,0
4,"Amor, teoricamente",Ali Hazelwood,9786555655254,5.0,NaN,Arqueiro,NaN,NaN,NaN,2023-08-21,NaN,NaN,NaN,0
5,Aliança de Casamento,Jasmine Guillory,9788542216431,2.5,NaN,Essência,NaN,NaN,NaN,2023-08-20,NaN,NaN,NaN,0
6,Le Petit Prince,Antoine de Saint-Exupéry,2070256626,4.0,NaN,Gallimard,NaN,NaN,NaN,2023-08-14,NaN,NaN,NaN,0
7,A Razão do Amor,Ali Hazelwood,9786555653991,5.0,NaN,Arqueiro,NaN,NaN,NaN,2023-08-14,NaN,NaN,NaN,0
8,ALEXANDER STONE,Cleo Luz,553380958,2.0,NaN,AMAZON,NaN,NaN,NaN,2023-08-08,NaN,NaN,NaN,0
9,Segredo lascivo,Natália Dias,553380958,2.5,NaN,Independente,NaN,NaN,NaN,2023-08-06,NaN,NaN,NaN,0


In [43]:
df_books['Media'].value_counts()

0    47
2     8
1     4
Name: Media, dtype: int64

In [46]:
df_books[df_books['Media'] == 2] #audioliveo

,Title,Author,ISBN,My Rating,Average Rating,Publisher,Binding,Year Published,Original Publication Year,Date Read,Date Added,Bookshelves,My Review,Media
1,Imperfeitos,Christina Lauren,9786559571284,3.5,NaN,Faro Editorial,NaN,NaN,NaN,2023-09-15,NaN,NaN,NaN,2
20,Essencialismo,Greg McKeown,8543102146,4.5,NaN,Sextante,NaN,NaN,NaN,2023-06-28,NaN,NaN,NaN,2
24,Abaixo de Zero,Ali Hazelwood,9786555654585,5.0,NaN,Arqueiro,NaN,NaN,NaN,2023-05-30,NaN,NaN,NaN,2
28,O Mais Desejado dos Highlanders,Maya Banks,553380958,3.0,NaN,Universo dos Livros,NaN,NaN,NaN,2023-05-21,NaN,NaN,NaN,2
29,Sob o mesmo teto,Ali Hazelwood,9786555654356,4.0,NaN,Arqueiro,NaN,NaN,NaN,2023-05-21,NaN,NaN,NaN,2
31,De olho em você,Amy Lea,9786555654332,5.0,NaN,Arqueiro,NaN,NaN,NaN,2023-05-05,NaN,NaN,NaN,2
50,A Última Fuga,Barbara Cartland,553380958,2.5,NaN,Editora Tecnoprint Ltda,NaN,NaN,NaN,2023-02-08,NaN,NaN,NaN,2
57,Clube do Livro dos Homens,Lyssa Kay Adams,6555650907,4.0,NaN,Arqueiro,NaN,NaN,NaN,2023-01-15,NaN,NaN,NaN,2


In [44]:
df_books[df_books['Media'] == 1] # ebook

,Title,Author,ISBN,My Rating,Average Rating,Publisher,Binding,Year Published,Original Publication Year,Date Read,Date Added,Bookshelves,My Review,Media
33,Doutor Arrogância,Bianca Pohndorf,553380958,2.0,NaN,Independente,NaN,NaN,NaN,2023-04-24,NaN,NaN,NaN,1
36,Meu Natal Com o Duque,Diane Bergher,553380958,3.0,NaN,Independente,NaN,NaN,NaN,2023-04-12,NaN,NaN,NaN,1
44,Um Acordo com o Duque,Flávia Padula,553380958,3.0,NaN,independente,NaN,NaN,NaN,2023-03-12,NaN,NaN,NaN,1
45,Onde Nascem os Sonhos,Lisa Kleypas,9786555653465,4.0,NaN,Arqueiro,NaN,NaN,NaN,2023-03-05,NaN,NaN,NaN,1


In [45]:
df_books[df_books['Media'] == 0] # papel

,Title,Author,ISBN,My Rating,Average Rating,Publisher,Binding,Year Published,Original Publication Year,Date Read,Date Added,Bookshelves,My Review,Media
0,"Tress, A Garota do Mar Esmeralda",Brandon Sanderson,9786589132943,5.0,NaN,Trama,NaN,NaN,NaN,2023-09-15,NaN,NaN,NaN,0
2,A Destruidora De Casamentos,Mia Sosa,9786559702664,4.0,NaN,Harlequin,NaN,NaN,NaN,2023-08-30,NaN,NaN,NaN,0
3,Conquistada pelo Bilionário Italiano,T. M. Kechichian,553380958,2.5,NaN,Independente,NaN,NaN,NaN,2023-08-27,NaN,NaN,NaN,0
4,"Amor, teoricamente",Ali Hazelwood,9786555655254,5.0,NaN,Arqueiro,NaN,NaN,NaN,2023-08-21,NaN,NaN,NaN,0
5,Aliança de Casamento,Jasmine Guillory,9788542216431,2.5,NaN,Essência,NaN,NaN,NaN,2023-08-20,NaN,NaN,NaN,0
6,Le Petit Prince,Antoine de Saint-Exupéry,2070256626,4.0,NaN,Gallimard,NaN,NaN,NaN,2023-08-14,NaN,NaN,NaN,0
7,A Razão do Amor,Ali Hazelwood,9786555653991,5.0,NaN,Arqueiro,NaN,NaN,NaN,2023-08-14,NaN,NaN,NaN,0
8,ALEXANDER STONE,Cleo Luz,553380958,2.0,NaN,AMAZON,NaN,NaN,NaN,2023-08-08,NaN,NaN,NaN,0
9,Segredo lascivo,Natália Dias,553380958,2.5,NaN,Independente,NaN,NaN,NaN,2023-08-06,NaN,NaN,NaN,0
10,Só Entre Nós Dois,Fernanda Santana,553380958,2.5,NaN,Independente,NaN,NaN,NaN,2023-08-03,NaN,NaN,NaN,0
